In [1]:
import numpy as np
import pywt
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import librosa

In [2]:
train_features_path = "../data/processed/balance_16k/preprocessing/x_train.csv"
labels_path = "../data/processed/balance_16k/preprocessing/y_train.csv"

In [3]:
df_train = pd.read_csv(train_features_path, header=None,nrows=100)
df_labels = pd.read_csv(labels_path, header=None,nrows=100)

In [4]:
df_train.columns = [str(i) for i in range(df_train.shape[1])]
df_labels.columns = [str(i) for i in range(df_labels.shape[1])]

In [5]:
def apply_cwt_to_single_row(row, num_columns_per_channel, scales, waveletname='morl'):
    BHZ = row[:num_columns_per_channel]         
    BHN = row[num_columns_per_channel:2*num_columns_per_channel] 
    BHE = row[2*num_columns_per_channel:]       
    
    cwt_BHZ, _ = pywt.cwt(BHZ, scales, waveletname)
    cwt_BHN, _ = pywt.cwt(BHN, scales, waveletname)
    cwt_BHE, _ = pywt.cwt(BHE, scales, waveletname)
    
    cwt_combined = np.stack([cwt_BHZ, cwt_BHN, cwt_BHE], axis=0)
    
    return cwt_combined

In [6]:
def apply_cwt_to_all_rows(data, num_columns_per_channel, scales, waveletname='morl'):
    cwt_results = []
    
    for _, row in data.iterrows():
        row_array = row.to_numpy()
        cwt_result = apply_cwt_to_single_row(row_array, num_columns_per_channel, scales, waveletname)
        cwt_results.append(cwt_result)
    
    return np.array(cwt_results)

In [7]:
df_combined = pd.concat([df_train, df_labels], axis=1)

In [8]:
columns = list(df_train.columns) + ['label']
df_combined.columns = columns

In [9]:
df_sampled = df_combined.groupby('label').apply(lambda x: x.sample(n=1, random_state=42)).reset_index(drop=True)


C:\Users\incognito\AppData\Local\Temp\ipykernel_23400\2257738727.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df_combined.groupby('label').apply(lambda x: x.sample(n=1, random_state=42)).reset_index(drop=True)


In [10]:
df_train_sampled = df_sampled.iloc[:, :-1]  
df_labels_sampled = df_sampled.iloc[:, -1] 

In [11]:
scales = np.arange(1, 128) 
num_columns_per_channel = 2401

cwt_transformed_data = apply_cwt_to_all_rows(df_train_sampled, num_columns_per_channel, scales)

In [12]:
def save_spectrogram_images_from_cwt(cwt_data, labels, output_folder):
    num_samples = cwt_data.shape[0]
    num_scales = cwt_data.shape[2]

    for i in range(num_samples):
        sample_data = cwt_data[i]
        label = labels[i]
        
        BHZ = np.array(sample_data[0])
        BHN = np.array(sample_data[1])
        BHE = np.array(sample_data[2]) 
        composite_image = np.zeros((num_scales, BHZ.shape[1], 3))
        composite_image[:, :, 0] = np.abs(BHZ)
        composite_image[:, :, 1] = np.abs(BHN)
        composite_image[:, :, 2] = np.abs(BHE)

        if np.isnan(composite_image).any() or np.isinf(composite_image).any():
            print(f"Error: NaN or Inf values detected in composite_image for sample {i}. Skipping this sample.")
            continue

        max_value = composite_image.max()
        if max_value != 0:
            composite_image /= max_value
        else:
            print(f"Warning: Max value is zero for sample {i}. Skipping normalization.")
            continue        
        filename = f"spectrogram_class_{label}_{i}.png"
        filepath = os.path.join(output_folder, filename)
        
        # Plot the spectrogram
        plt.figure(figsize=(10, 6))
        plt.imshow(composite_image, aspect='auto')
        plt.title(f'Spectrogram Image (Class {label})')
        plt.xlabel('Time')
        plt.ylabel('Scales')
        plt.colorbar(label='Amplitude')  # Commented for RGB images
        
        # Save the image
        plt.savefig(filepath, format='png')  # Ensure png format
        plt.close()  # Close the plot to avoid memory issues

        # print(f"Saved: {filepath}")

# Example usage
output_folder = "../spectrogram_images/"
save_spectrogram_images_from_cwt(cwt_transformed_data, df_labels_sampled, output_folder)